In [1]:
import pandas as pd
import datetime
from datetime import datetime
from time import strptime


In [2]:
#Reading the csv file ("Rentals" + lattitudes & longitudes)
a = pd.read_csv("Pittsburgh_dataset_lon_lat.csv", header = 0)

In [3]:
#Analyzing the merged df
a.head(1)

,trip_id,start_time,stop_time,bike_id,trip_duration,from_station_id,from_station_name,to_station_id,to_station_name,user_type,from_latitude,from_longitude,to_latitude,to_longitude
0,10676972,2015-05-31 06:54:00,2015-05-31 10:17:00,70166,12190,1018,Butler St & Stanton Ave,1001,Forbes Ave & Market Square,Customer,40.466103,-79.964628,40.440877,-80.00308


In [15]:
#Historical daily weather data is downloaded as CSV format from NOAA website as transmitted by "Pittsburgh International Airport" weather station
b = pd.read_csv("Pittsburg Airport.csv", header = 0, parse_dates = True)

In [5]:
"""
The columns included in this dataset are:
WSF5 - Fastest 5-second wind speed
WT03 - Thunder
WT04 - Ice pellets, sleet, snow pellets, or small hail" 
PRCP - Precipitation
WT08 - Smoke or haze 
WT09 - Blowing or drifting snow
WDF2 - Direction of fastest 2-minute wind
AWND - Average wind speed
WDF5 - Direction of fastest 5-second wind
PGTM - Peak gust time
WT01 - Fog, ice fog, or freezing fog (may include heavy fog)
TMAX - Maximum temperature
WT02 - Heavy fog or heaving freezing fog (not always distinguished from fog)
TAVG - Average Temperature.
TMIN - Minimum temperature
WSF2 - Fastest 2-minute wind speed"""
b.head(2)

,STATION,NAME,DATE,AWND,PGTM,PRCP,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT03,WT04,WT08,WT09
0,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",2015-04-01,4.70,1443,0.00,NaN,58.0,28.0,270,290,14.1,19.0,NaN,NaN,NaN,NaN,NaN,NaN
1,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",2015-04-02,11.41,1202,0.18,NaN,69.0,37.0,240,260,35.1,45.0,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#I wanted to retain only columns such as average precipitation, maximum temperature,date,Minimum temperature,etc.,  using Excel.
c = b.drop(["STATION","NAME","PGTM","TAVG","WDF2","WDF5","WSF2","WSF5"], axis =1)


In [7]:
c['DATE'] = pd.to_datetime(c['DATE'], format =  '%Y-%m-%d')

In [8]:
c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371 entries, 0 to 1370
Data columns (total 11 columns):
DATE    1371 non-null datetime64[ns]
AWND    1368 non-null float64
PRCP    1371 non-null float64
TMAX    1369 non-null float64
TMIN    1369 non-null float64
WT01    47 non-null float64
WT02    7 non-null float64
WT03    2 non-null float64
WT04    1 non-null float64
WT08    23 non-null float64
WT09    1 non-null float64
dtypes: datetime64[ns](1), float64(10)
memory usage: 117.9 KB


In [9]:
#To merge weather data to The Bikeshare rentals, We need just the datefield from the "Start_time" column , because it is in datetime format while the weather data’s date field is in date format. 
#To aid in merging dataframes, I created a temporary "DATE" column by stripping the time part of the date in "Renatals" df
a['DATE'] = pd.to_datetime(a['start_time']).dt.date

In [11]:
a.tail(1)

,trip_id,start_time,stop_time,bike_id,trip_duration,from_station_id,from_station_name,to_station_id,to_station_name,user_type,from_latitude,from_longitude,to_latitude,to_longitude,DATE
269686,55956393,2018-08-13 17:33:00,2018-08-13 18:09:00,70428,2154,1061,33rd St and Penn Ave,1042,Centre Ave & Kirkpatrick St,Subscriber,40.462026,-79.968114,40.445019,-79.977194,2018-08-13


In [12]:
#  Here, I am extracting only the month and day from a timestamp using the datetime module  and then segregating
#seasons based on whether it falls within a given season (fall, summer, winter, spring) based on the fixed dates of the solstices and equinoxes 
def season_of_date(date):
    year = str(date.year)
    seasons = {'spring': pd.date_range(start='21/03/'+year, end='20/06/'+year),
               'summer': pd.date_range(start='21/06/'+year, end='22/09/'+year),
               'autumn': pd.date_range(start='23/09/'+year, end='20/12/'+year)}
    if date in seasons['spring']:
        return 'spring'
    if date in seasons['summer']:
        return 'summer'
    if date in seasons['autumn']:
        return 'autumn'
    else:
        return 'winter'
# Map the results to "DATE" column of type `datetime`
c['season'] = c.DATE.map(season_of_date)

In [13]:
c.head(2)

,DATE,AWND,PRCP,TMAX,TMIN,WT01,WT02,WT03,WT04,WT08,WT09,season
0,2015-04-01,4.70,0.00,58.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,spring
1,2015-04-02,11.41,0.18,69.0,37.0,NaN,NaN,NaN,NaN,NaN,NaN,spring


In [14]:
c.to_csv("Pi_seasons.csv",index = False)